In [2]:
import os
import json
import shutil
from pathlib import Path
from ultralytics import YOLO
import yaml
from tqdm import tqdm
import random


In [ ]:

# --- 1. 配置参数 ---
CONFIG = {
    "model_size": "yolov8m",       # yolov8n/s/m/l/x (x最大最准确)
    "img_size": 1280,              # 输入图像尺寸
    "batch_size": 8,               # 根据显存调整
    "epochs": 30,                 # 训练轮数
    "patience": 15,                # 早停耐心值
    "workers": 4,                  # 数据加载线程数
    "device": 0,                   # GPU编号
    
    # 数据路径
    "train_img_dir": "train",
    "train_json": "train.json",
    
    # 输出路径
    "dataset_root": "yolo_dataset",
    "project_name": "road_hazard_yolov8",
    
    # 数据集划分（从训练集中分出验证集）
    "val_split_ratio": 0.15,      # 15%作为验证集
    "random_seed": 42,             # 随机种子，保证可复现
}

# 类别映射
CATEGORY_MAPPING = {
    1: 0,  # Pothole -> 0
    2: 1,  # Net -> 1
}

CLASS_NAMES = ["Pothole", "Net"]

# --- 2. COCO to YOLO格式转换（改进版：支持训练/验证集划分）---
def convert_coco_to_yolo_with_split(coco_json_path, img_dir, output_dir, val_split_ratio=0.15):
    """
    将COCO格式转换为YOLO格式，并自动划分训练集和验证集
    """
    print(f"\n{'='*60}")
    print(f"📝 正在转换数据集为YOLO格式...")
    print(f"{'='*60}")
    
    # 创建输出目录
    for split in ['train', 'val']:
        (Path(output_dir) / "images" / split).mkdir(parents=True, exist_ok=True)
        (Path(output_dir) / "labels" / split).mkdir(parents=True, exist_ok=True)
    
    # 加载COCO标注
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)
    
    # 建立image_id到标注的映射
    img_id_to_anns = {}
    for ann in coco_data['annotations']:
        img_id = ann['image_id']
        if img_id not in img_id_to_anns:
            img_id_to_anns[img_id] = []
        img_id_to_anns[img_id].append(ann)
    
    # 随机划分训练集和验证集
    random.seed(CONFIG['random_seed'])
    all_images = coco_data['images']
    random.shuffle(all_images)
    
    val_size = int(len(all_images) * val_split_ratio)
    val_images = all_images[:val_size]
    train_images = all_images[val_size:]
    
    print(f"\n📊 数据集划分:")
    print(f"   总图片数: {len(all_images)}")
    print(f"   训练集: {len(train_images)} ({100*(1-val_split_ratio):.1f}%)")
    print(f"   验证集: {len(val_images)} ({100*val_split_ratio:.1f}%)")
    
    # 处理训练集和验证集
    stats = {'train': {'success': 0, 'no_ann': 0}, 
             'val': {'success': 0, 'no_ann': 0}}
    
    for split, images in [('train', train_images), ('val', val_images)]:
        print(f"\n🔄 正在处理 {split} 集...")
        
        for img_info in tqdm(images, desc=f"转换{split}集"):
            img_id = img_info['id']
            file_name = img_info['file_name']
            img_width = img_info['width']
            img_height = img_info['height']
            
            # 复制图片
            src_img_path = Path(img_dir) / file_name
            if not src_img_path.exists():
                print(f"⚠️ 警告: 图片不存在 {src_img_path}")
                continue
            
            dst_img_path = Path(output_dir) / "images" / split / file_name
            shutil.copy(src_img_path, dst_img_path)
            
            # 转换标注
            label_file = Path(output_dir) / "labels" / split / (Path(file_name).stem + ".txt")
            
            anns = img_id_to_anns.get(img_id, [])
            
            if len(anns) == 0:
                # 创建空标注文件（负样本）
                label_file.write_text("")
                stats[split]['no_ann'] += 1
            else:
                yolo_lines = []
                for ann in anns:
                    bbox = ann['bbox']
                    category_id = ann['category_id']
                    
                    if category_id not in CATEGORY_MAPPING:
                        continue
                    
                    class_id = CATEGORY_MAPPING[category_id]
                    
                    # 计算YOLO格式坐标
                    x_center = (bbox[0] + bbox[2] / 2) / img_width
                    y_center = (bbox[1] + bbox[3] / 2) / img_height
                    width = bbox[2] / img_width
                    height = bbox[3] / img_height
                    
                    # 边界检查
                    x_center = max(0, min(1, x_center))
                    y_center = max(0, min(1, y_center))
                    width = max(0, min(1, width))
                    height = max(0, min(1, height))
                    
                    yolo_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
                
                if yolo_lines:
                    label_file.write_text("\n".join(yolo_lines))
                    stats[split]['success'] += 1
        
        print(f"✅ {split}集转换完成:")
        print(f"   - 有标注图片: {stats[split]['success']} 张")
        print(f"   - 无标注图片: {stats[split]['no_ann']} 张")
        print(f"   - 总计: {stats[split]['success'] + stats[split]['no_ann']} 张")

# --- 3. 创建YOLO数据集配置文件 ---
def create_yaml_config(dataset_root, class_names):
    """创建dataset.yaml配置文件"""
    yaml_path = Path(dataset_root) / "dataset.yaml"
    
    config = {
        'path': str(Path(dataset_root).absolute()),
        'train': 'images/train',
        'val': 'images/val',
        'names': {i: name for i, name in enumerate(class_names)},
        'nc': len(class_names),
    }
    
    with open(yaml_path, 'w') as f:
        yaml.dump(config, f, sort_keys=False)
    
    print(f"\n✅ 配置文件已创建: {yaml_path}")
    return yaml_path

# --- 4. 主训练流程 ---
def main():
    print(f"\n{'='*70}")
    print(f"🚀 YOLOv8 路面缺陷检测训练系统（自动划分验证集）")
    print(f"{'='*70}")
    
    print(f"\n📊 训练配置:")
    print(f"   模型: {CONFIG['model_size'].upper()}")
    print(f"   图像尺寸: {CONFIG['img_size']}px")
    print(f"   训练轮数: {CONFIG['epochs']} epochs")
    print(f"   Batch Size: {CONFIG['batch_size']}")
    print(f"   验证集比例: {CONFIG['val_split_ratio']*100:.0f}%")
    
    # 1. 检查训练数据
    if not Path(CONFIG['train_json']).exists():
        print(f"\n❌ 错误: 找不到训练标注文件 {CONFIG['train_json']}")
        return
    
    if not Path(CONFIG['train_img_dir']).exists():
        print(f"\n❌ 错误: 找不到训练图片目录 {CONFIG['train_img_dir']}")
        return
    
    # 2. 数据格式转换（自动划分训练集/验证集）
    convert_coco_to_yolo_with_split(
        CONFIG['train_json'],
        CONFIG['train_img_dir'],
        CONFIG['dataset_root'],
        val_split_ratio=CONFIG['val_split_ratio']
    )
    
    # 3. 创建配置文件
    yaml_path = create_yaml_config(CONFIG['dataset_root'], CLASS_NAMES)
    
    # 4. 初始化YOLO模型
    print(f"\n{'='*60}")
    print(f"🏗️ 正在初始化 {CONFIG['model_size'].upper()} 模型...")
    print(f"{'='*60}")
    
    model = YOLO(f"{CONFIG['model_size']}.pt")
    
    # 5. 开始训练
    print(f"\n{'='*60}")
    print(f"🎓 开始训练...")
    print(f"{'='*60}\n")
    
    results = model.train(
        data=str(yaml_path),
        epochs=CONFIG['epochs'],
        imgsz=CONFIG['img_size'],
        batch=CONFIG['batch_size'],
        
        # 优化器配置
        optimizer='AdamW',
        lr0=0.001,
        lrf=0.01,
        momentum=0.937,
        weight_decay=0.0005,
        
        # 数据增强
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,
        degrees=10.0,
        translate=0.1,
        scale=0.5,
        shear=0.0,
        perspective=0.0,
        flipud=0.0,
        fliplr=0.5,
        mosaic=1.0,
        mixup=0.15,
        copy_paste=0.3,
        
        # 训练策略
        patience=CONFIG['patience'],
        save=True,
        save_period=10,
        workers=CONFIG['workers'],
        device=CONFIG['device'],
        
        # 其他配置
        project=CONFIG['project_name'],
        name='train',
        exist_ok=True,
        pretrained=True,
        verbose=True,
        seed=CONFIG['random_seed'],
        
        # 高级配置
        cos_lr=True,
        close_mosaic=10,
        amp=True,
        fraction=1.0,
        
        # 损失权重（针对小目标优化）
        box=7.5,
        cls=0.5,
        dfl=1.5,
    )
    
    # 6. 训练完成
    print(f"\n{'='*60}")
    print(f"🎉 训练完成!")
    print(f"{'='*60}")
    print(f"\n📁 训练结果保存在: {CONFIG['project_name']}/train/")
    print(f"🏆 最佳模型: {CONFIG['project_name']}/train/weights/best.pt")
    print(f"📈 最新模型: {CONFIG['project_name']}/train/weights/last.pt")
    
    # 7. 验证模型
    print(f"\n{'='*60}")
    print(f"🔍 正在验证模型性能（内部验证集）...")
    print(f"{'='*60}\n")
    
    best_model = YOLO(f"{CONFIG['project_name']}/train/weights/best.pt")
    metrics = best_model.val(
        data=str(yaml_path),
        imgsz=CONFIG['img_size'],
        batch=CONFIG['batch_size'],
        conf=0.001,
        iou=0.6,
        device=CONFIG['device'],
    )
    
    # 打印关键指标
    print(f"\n📊 验证集性能指标:")
    print(f"   mAP50: {metrics.box.map50:.4f}")
    print(f"   mAP50-95: {metrics.box.map:.4f}")
    print(f"   Precision: {metrics.box.mp:.4f}")
    print(f"   Recall: {metrics.box.mr:.4f}")
    
    print(f"\n✅ 所有任务完成！")
    print(f"💡 提示: 现在可以运行 inference_yolov8.py 对真实验证集进行推理")

if __name__ == '__main__':
    main()


🚀 YOLOv8 路面缺陷检测训练系统（自动划分验证集）

📊 训练配置:
   模型: YOLOV8M
   图像尺寸: 1280px
   训练轮数: 30 epochs
   Batch Size: 8
   验证集比例: 15%

📝 正在转换数据集为YOLO格式...

📊 数据集划分:
   总图片数: 3000
   训练集: 2550 (85.0%)
   验证集: 450 (15.0%)

🔄 正在处理 train 集...


转换train集: 100%|██████████| 2550/2550 [00:03<00:00, 816.40it/s]


✅ train集转换完成:
   - 有标注图片: 2550 张
   - 无标注图片: 0 张
   - 总计: 2550 张

🔄 正在处理 val 集...


转换val集: 100%|██████████| 450/450 [00:00<00:00, 893.95it/s]


✅ val集转换完成:
   - 有标注图片: 450 张
   - 无标注图片: 0 张
   - 总计: 450 张

✅ 配置文件已创建: yolo_dataset\dataset.yaml

🏗️ 正在初始化 YOLOV8M 模型...

🎓 开始训练...

Ultralytics 8.3.241  Python-3.12.12 torch-2.9.1+cu126 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.3, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=yolo_dataset\dataset.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.15, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False,

V3

In [33]:
import os
import json
import random
import shutil
from pathlib import Path
from tqdm import tqdm
import yaml
from ultralytics import YOLO

# --- 1. V3 高级微调配置 ---
CONFIG = {
    # 🎯 依然基于 V1 的最佳模型开始 (也可以尝试用 V2 的 result，但 V1 的泛化性可能更好作为基座)
    "existing_model_path": "road_hazard_yolov8/train/weights/best.pt", 
    
    # 📁 数据集路径 (保持不变)
    "dataset_root": "yolo_dataset",
    
    # 🔧 训练参数
    "img_size": 1280,
    "batch_size": 8,               # 显存允许的话，尽量大一点
    "epochs": 40,                  # 解冻训练需要更多轮次来适应
    "workers": 4,
    "device": 0,
    
    # 📤 输出配置
    "project_name": "road_hazard_yolov8_finetune",
    "run_name": "finetune_v3_unfrozen",  # V3 版本名称
    
    "class_names": ["Pothole", "Net"]
}

# --- 2. 数据集检查 (安全锁) ---
def check_dataset_safe(dataset_root):
    path = Path(dataset_root) / "labels/train"
    if not path.exists():
        print(f"❌ 严重错误: 数据集 {dataset_root} 不存在！")
        print("请务必使用第一次训练生成的 dataset，不要重新生成，否则会数据泄露！")
        exit()
    print(f"✅ 数据集安全检查通过: {dataset_root}")

# --- 3. YAML 配置生成 ---
def create_yaml_config(dataset_root, class_names):
    yaml_path = Path(dataset_root) / "dataset.yaml"
    config = {
        'path': str(Path(dataset_root).absolute()),
        'train': 'images/train',
        'val': 'images/val',
        'names': {i: name for i, name in enumerate(class_names)},
        'nc': len(class_names),
    }
    with open(yaml_path, 'w') as f:
        yaml.dump(config, f, sort_keys=False)
    return yaml_path

# --- 4. 主训练流程 (V3 激进模式) ---
def main():
    print(f"\n{'='*70}")
    print(f"🚀 YOLOv8 V3 深度微调 (Unfrozen + AdamW)")
    print(f"{'='*70}")
    
    # 1. 检查
    if not Path(CONFIG['existing_model_path']).exists():
        print(f"❌ 找不到模型: {CONFIG['existing_model_path']}")
        return
    check_dataset_safe(CONFIG['dataset_root'])
    
    # 2. 配置
    yaml_path = create_yaml_config(CONFIG['dataset_root'], CONFIG['class_names'])
    
    # 3. 加载模型
    print(f"\n🏗️ 加载基座模型: {CONFIG['existing_model_path']}")
    model = YOLO(CONFIG['existing_model_path'])
    
    # 4. 开始 V3 训练
    print(f"\n🔥 启动 V3 训练策略:")
    print(f"   1. 全网解冻 (Freeze=0) - 适应路面纹理")
    print(f"   2. 优化器 AdamW - 更精细的权重更新")
    print(f"   3. 增强几何/光照变换 - 提升泛化性")
    print(f"   4. 关闭 Mosaic - 保持真实尺度")
    
    results = model.train(
        data=str(yaml_path),
        epochs=CONFIG['epochs'],
        imgsz=CONFIG['img_size'],
        batch=CONFIG['batch_size'],
        
        # ✅ V3 核心优化策略
        optimizer='AdamW',    # 更改优化器
        lr0=0.0002,           # AdamW 的初始学习率通常比 SGD 低
        lrf=0.05,             # 最终学习率衰减 (保持一定的更新能力)
        cos_lr=True,          # 余弦退火，平滑收敛
        
        # ✅ 深度适配策略
        freeze=0,             # 【关键】解冻所有层，让 Backbone 适应路面
        
        # ✅ 数据增强 (模拟真实路况)
        mosaic=0.0,           # 保持关闭，不破坏小目标完整性
        mixup=0.1,            # 轻微混合，增加鲁棒性
        copy_paste=0.0,       # 关闭
        
        # 几何变换 (模拟车载视角)
        degrees=2.0,          # 轻微旋转
        translate=0.1,        # 平移
        scale=0.3,            # 尺度变化 (远近/大小)
        shear=1.0,            # 剪切变换 (透视效果)
        perspective=0.0001,   # 极其轻微的透视
        flipud=0.0,           # 路面通常不会上下颠倒
        fliplr=0.5,           # 左右翻转
        
        # 光照变换 (模拟天气)
        hsv_h=0.015,          # 色调
        hsv_s=0.7,            # 饱和度 (路面干湿差异)
        hsv_v=0.4,            # 亮度 (阴影/阳光)
        
        # 常规设置
        patience=12,          # 稍微宽容一点的早停
        save=True,
        workers=CONFIG['workers'],
        device=CONFIG['device'],
        project=CONFIG['project_name'],
        name=CONFIG['run_name'],
        exist_ok=True,
        pretrained=True,
        
        # 损失权重 (针对小目标优化)
        box=8.0,              # 稍微增加边框回归权重，要求定位更准
        cls=0.5,
        dfl=1.5,
    )
    
    print(f"\n{'='*60}")
    print(f"🎉 V3 训练完成！")
    print(f"📁 结果路径: {CONFIG['project_name']}/{CONFIG['run_name']}")
    print(f"🏆 最佳模型: {CONFIG['project_name']}/{CONFIG['run_name']}/weights/best.pt")
    print(f"{'='*60}")
    
    # 5. 验证
    print("\n🔍 验证 V3 模型性能...")
    metrics = model.val(
        data=str(yaml_path),
        split='val',
        imgsz=CONFIG['img_size'],
        batch=CONFIG['batch_size'],
        conf=0.001,
        iou=0.6,
        device=CONFIG['device']
    )
    
    print(f"\n📊 V3 验证集指标:")
    print(f"   mAP50: {metrics.box.map50:.4f}")
    print(f"   mAP50-95: {metrics.box.map:.4f}")

if __name__ == '__main__':
    main()


🚀 YOLOv8 V3 深度微调 (Unfrozen + AdamW)
✅ 数据集安全检查通过: yolo_dataset

🏗️ 加载基座模型: road_hazard_yolov8/train/weights/best.pt

🔥 启动 V3 训练策略:
   1. 全网解冻 (Freeze=0) - 适应路面纹理
   2. 优化器 AdamW - 更精细的权重更新
   3. 增强几何/光照变换 - 提升泛化性
   4. 关闭 Mosaic - 保持真实尺度
Ultralytics 8.3.241  Python-3.12.12 torch-2.9.1+cu126 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=8.0, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=yolo_dataset\dataset.yaml, degrees=2.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0002, lrf=0.05, mask_ratio=4, m